In [2]:
# @hidden cell
# load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

import os
import sys

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

%aimport features
from features.generate_features import access_spotify, get_song_ids, get_features
import pandas as pd
import matplotlib.pyplot as plt
import seaborn

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
